In [ ]:
# Over the course of this tutorial I will refer to solar images as maps, or a set of them as mapsequence.
# Beware that the peek function even though provides a nice interactive way to view your maps but clogs your memory.
# As of now I still havent found a way to clear the figures from the memory. So the more you use the peek function the slower the code will get.
# You can avoid it by using using plot instead of peek but it will be lot less interactive .
# If the code gets very slow just restart the kernal start again with your new insights .

In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from glob import glob
from sunpy.map import Map
from datetime import datetime,timedelta
import seaborn as sns

import astropy.units as u
from astropy.coordinates import SkyCoord
from matplotlib.path import Path
from sunpy import timeseries as ts

from sunkit_image.coalignment import calculate_match_template_shift as cal_shift
from sunkit_image.coalignment import apply_shifts 

from matplotlib.lines import Line2D
from matplotlib.colors import PowerNorm
import csv
from sunpy import timeseries as ts

$\huge \text{Next up we have a couple of functions that will help us along the way.}$  

Python functions are named, reusable blocks of code designed to perform specific tasks.  

They promote modularity, code reuse, and easier maintenance in programming.  


In [ ]:
# This function exposure normalises your data (new data = old data/exposure ) and changes the header too.

def exp_norm(map_sequence):
    mod_rois = []
    for smap in tqdm(map_sequence):
        esf = (smap.meta['cmd_expt'])/1e3 #exposure scaling factor
        data = smap.data
        data = data/esf   
        smap.meta['cmd_expt'] = 1000 #changing the exposure to 1 sec
        mod_rois.append(Map(data,smap.meta))
    mod_rois = Map(mod_rois,sequence=True)
    return mod_rois

In [3]:
# This function will prompt you to select corners and will return the resulting quadilateral as a map.

def submap_with_ginput(sunpy_map):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection=sunpy_map)
    sunpy_map.plot(axes=ax)
    ax.set_title("Click two corners of the ROI (bottom-left and top-right)")

    # Wait for 2 clicks
    pts = plt.ginput(2, timeout=0)
    plt.close(fig)

    if len(pts) != 2:
        raise RuntimeError("ROI selection cancelled or failed.")

    (x1, y1), (x2, y2) = pts
    bottom_left = (min(x1, x2), min(y1, y2)) * u.pixel
    top_right = (max(x1, x2), max(y1, y2)) * u.pixel

    submap = sunpy_map.submap(bottom_left=bottom_left, top_right=top_right)
    return submap

In [4]:
# This function takes your map sequence as input and returns coaligned maps as output.
# Check Coalignment module for more info. 
# Vist *https://docs.sunpy.org/projects/sunkit-image/en/latest/code_ref/coalignment.html* to know more about the functions used.

def co_align(map_sequence,template,layer_index=0,clip=False):
    shifts = cal_shift(map_sequence,layer_index=layer_index,template=template)
    plate_scale = map_sequence[0].scale[0]
    if clip is False:
        coaligned_maps = apply_shifts(map_sequence, xshift=-shifts['x']/plate_scale, yshift=-shifts['y']/plate_scale, clip=False)
    if clip is True:
        coaligned_maps = apply_shifts(map_sequence, xshift=-shifts['x']/plate_scale, yshift=-shifts['y']/plate_scale, clip=True)
    return coaligned_maps

In [ ]:
# crops the maps in the mapsequence using the corners of the sample map provided.

def crop_maps(mc,sample):
    z=[]
    b_left =  sample.bottom_left_coord
    t_right = sample.top_right_coord
    x1,y1 = sample.world_to_pixel(b_left)
    x2,y2 = sample.world_to_pixel(t_right)
    for temp in mc:
        z.append(temp.submap(bottom_left = [x1,y1]*u.pix,top_right = [x2,y2]*u.pix))  
    z=Map(z,sequence=True)
    return z

In [ ]:
# This function uses the co_align and submap_with_ginput function internally to align your mapsequence and plots the aligned maps. 
# Function also prints the coordinates of the selected template.

def align(mc,index=0):
    
    templ = submap_with_ginput(mc[index])
    fig=plt.figure()
    ax=fig.add_subplot(111,projection=templ)
    templ.plot(axes=ax)
    ax.set_title("Click anywhere to close the template and begin coalignment",fontweight="bold")
    
    plt.waitforbuttonpress()
    plt.close()
    print(templ.bottom_left_coord,templ.top_right_coord)

    aligned = co_align(map_sequence=mc,layer_index=0,template=templ,clip=False)
    aligned.peek();plt.show();plt.colorbar()
    return aligned

In [ ]:
# function to make lightcurve by taking mean of unsaturated pixels inside the mask for the entire mapsequence.

def make_lc(mask,map_sequence):
    lc=[];time=[]
    for smap in map_sequence:
        data = smap.data
        lc.append(np.nanmean(data[mask]))  #we have earlier put saturating pixels to nan so need to use nanmean here.
        time.append(datetime.strptime(smap.meta['DHOBT_DT'].split('.')[0],"%Y-%m-%dT%H:%M:%S")) 
        #output = ['2024-11-10T13:59:58', '511920000'] to ignore the nanosec part you only take index 0
        
    return time,lc

In [2]:
def bin_down(map_sequence,saturation_lvl=55000):
    unsaturated_maps = []
    dim = [2,2]
    for smap in map_sequence:
        data = smap.data
        data[data>saturation_lvl] = np.nan #replace the saturating pixels with nan, so that even after binning, they don't affect the data.
        temp = Map(data,smap.meta)
        unsaturated_maps.append(temp.superpixel(dim*u.pix))
    unsaturated_maps = Map(unsaturated_maps,sequence=True)
    # we bin 2x2 pixel to make one superpixel. If even one of the pixels in the 2x2 pixels is saturating, we replace the entire superpixel by nan.
    # Now when we coalign we cannot have nan in the data, so we replace the nans by some fixed -ve value, let's say -1000. which we can ignore when making light curves.
    final_maps = []
    for smap in unsaturated_maps:
        data=smap.data
        data = np.nan_to_num(A, nan=-1000)
        final_maps.append(Map(data,smap.meta))
    final_maps = Map(final_maps,sequence=True)
    return final_maps

$\huge \text{We start by loading data for different filters as sunpy maps}$

In [ ]:
# Lets start with NB03
basepath = 'data/'

# loading all the NB03 files
files3 = sorted(glob(basepath+'NB03/*NB03.fits'))

# generate a mapsequence.
smaps3 = bin_down(Map(files3,sequence=True))

#sometimes the data might not be of the same size. Hence we crop the remaining data to the size of the smaps3[0]
smaps3 = crop_maps(smaps3,smaps3[0])  

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:01<00:00, 28.28it/s]


In [ ]:
# Loading rest of the dataset
files1 = sorted(glob(basepath+'NB01/*NB01.fits'))
smaps1 = crop_maps(bin_down(Map(files1,sequence=True)),smaps3[0])

files2 = sorted(glob(basepath+'NB02/*NB02.fits'))
smaps2 = crop_maps(bin_down(Map(files2,sequence=True)),smaps3[0])

files4 = sorted(glob(basepath+'NB04/*NB04.fits'))
smaps4 = crop_maps(bin_down(Map(files4,sequence=True)),smaps3[0])

files5 = sorted(glob(basepath+'NB05/*NB05.fits'))
smaps5 = crop_maps(bin_down(Map(files5,sequence=True)),smaps3[0])

files6 = sorted(glob(basepath+'NB06/*NB06.fits'))
smaps6 = crop_maps(bin_down(Map(files6,sequence=True)),smaps3[0])

files7 = sorted(glob(basepath+'NB07/*NB07.fits'))
smaps7 = crop_maps(bin_down(Map(files7,sequence=True)),smaps3[0])

files8 = sorted(glob(basepath+'NB08/*NB08.fits'))
smaps8 = crop_maps(bin_down(Map(files8,sequence=True)),smaps3[0])

files9 = sorted(glob(basepath+'BB01/*BB01.fits'))
smaps9 = crop_maps(bin_down(Map(files9,sequence=True)),smaps3[0])

files10 = sorted(glob(basepath+'BB02/*BB02.fits'))
smaps10 = crop_maps(bin_down(Map(files10,sequence=True)),smaps3[0])

files11 = sorted(glob(basepath+'BB03/*BB03.fits'))
smaps11 = crop_maps(bin_down(Map(files11,sequence=True)),smaps3[0])

In [ ]:
# Have a look at any map u like.
smaps2.peek()

$\huge \text{Align first image of each filter with each other.}$

In [ ]:
# make a map sequence of first images of all the different filters.

temp = Map(smaps1[0],smaps2[0],smaps3[0],smaps4[0],smaps5[0],smaps6[0],smaps7[0],smaps8[0],
           smaps9[0],smaps10[0],smaps11[0],sequence=True)
temp.peek();plt.show();plt.colorbar()

filter_index = {}
for i,tmap in enumerate(temp):
    filter_index[f"{tmap.meta['ftr_name']}"] = i
print(filter_index) # to see the filter indices

In [ ]:
# Now we create template to coalign our mapsequence. I prefer using NB03 to make a template.
# For more tips on coalignment, look up the coalignment module.

template = submap_with_ginput(smaps3[0])
template.plot()

In [ ]:
# align the images to the NB03 image using the template
index =  # we are using 5 because it corresponds to NB03 map in the map sequence. You can choose others also.It just aligns everything to that particular frame
temp = co_align(temp,layer_index=index,template=template)
temp.peek();plt.show()

In [ ]:
# if the coalignment does not work in the above cell, use this template to coalign

# bottom_left = SkyCoord(-407*u.arcsec, 397*u.arcsec, frame=smaps3[0].coordinate_frame)
# top_right = SkyCoord(-348*u.arcsec, 442*u.arcsec, frame=smaps3[0].coordinate_frame)
# template1 = smaps3[0].submap(bottom_left,top_right=top_right)
# template1.peek()

In [ ]:
# replace the first map of the different filters with the one aligned with NB03

smaps1.maps[0]  = temp[filter_index[f"{smaps1[1].meta['ftr_name']}"]]
smaps2.maps[0]  = temp[filter_index[f"{smaps2[1].meta['ftr_name']}"]]
smaps3.maps[0]  = temp[filter_index[f"{smaps3[1].meta['ftr_name']}"]]
smaps4.maps[0]  = temp[filter_index[f"{smaps4[1].meta['ftr_name']}"]]
smaps5.maps[0]  = temp[filter_index[f"{smaps5[1].meta['ftr_name']}"]]
smaps6.maps[0]  = temp[filter_index[f"{smaps6[1].meta['ftr_name']}"]]
smaps7.maps[0]  = temp[filter_index[f"{smaps7[1].meta['ftr_name']}"]]
smaps8.maps[0]  = temp[filter_index[f"{smaps8[1].meta['ftr_name']}"]]
smaps9.maps[0]  = temp[filter_index[f"{smaps9[1].meta['ftr_name']}"]]
smaps10.maps[0] = temp[filter_index[f"{smaps10[1].meta['ftr_name']}"]]
smaps11.maps[0] = temp[filter_index[f"{smaps11[1].meta['ftr_name']}"]]

$\huge \text{  Now we align all the images of a particular filter to the first image.}$
$\huge \text{  "align" function will make a template for you and then align all images to the first image using that template.}$ 
$\huge \text{  This might take a few tries.}$

$\huge \text{  Keep trying until you are confident with the coalignment. Have a look at the coalignment module for tips on selecting a template.}$


In [ ]:
# The align function will ask you to select bottom left and top right corner to select a template.
# It then shows you the template and waits for you to click and then starts coalignment.
# After the data is coaligned, it gets plotted for you to have a look.

coaligned_maps1 = align(smaps1)

##################################################################################
# align function also prints the bottom_left and top_right coordinates of the selected template.
# once you are happy with the alignment, you might want to save those coordinates and use them to coalign without ginput as shown below.
# so after getting the coordinates of the best template uncomment the below code and insert the coordinates run the cell again.

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps1[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps1[0].coordinate_frame)
# template = smaps1[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps1 = co_align(smaps1,layer_index=0,template=template,clip=False)
# coaligned_maps1.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps2 = align(smaps2)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps1[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps1[0].coordinate_frame)
# template = smaps1[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps1 = co_align(smaps1,layer_index=0,template=template,clip=False)
# coaligned_maps1.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps3 = align(smaps3)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps3[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps3[0].coordinate_frame)
# template = smaps3[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps3 = co_align(smaps3,layer_index=0,template=template,clip=False)
# coaligned_maps3.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps4 = align(smaps4)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps4[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps4[0].coordinate_frame)
# template = smaps4[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps4 = co_align(smaps4,layer_index=0,template=template,clip=False)
# coaligned_maps4.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps5 = align(smaps5)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps5[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps5[0].coordinate_frame)
# template = smaps5[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps5 = co_align(smaps5,layer_index=0,template=template,clip=False)
# coaligned_maps5.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps6 = align(smaps6)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps6[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps6[0].coordinate_frame)
# template = smaps6[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps6 = co_align(smaps6,layer_index=0,template=template,clip=False)
# coaligned_maps6.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps7 = align(smaps7)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps7[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps7[0].coordinate_frame)
# template = smaps7[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps7 = co_align(smaps7,layer_index=0,template=template,clip=False)
# coaligned_maps7.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps8 = align(smaps8)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps8[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps8[0].coordinate_frame)
# template = smaps8[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps8 = co_align(smaps8,layer_index=0,template=template,clip=False)
# coaligned_maps8.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps9 = align(smaps9)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps9[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps9[0].coordinate_frame)
# template = smaps9[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps9 = co_align(smaps9,layer_index=0,template=template,clip=False)
# coaligned_maps9.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps10 = align(smaps10)

# b_left = SkyCoord(*u.arcsec,   *u.arcsec, frame=smaps10[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps10[0].coordinate_frame)
# template = smaps10[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps10 = co_align(smaps10,layer_index=0,template=template,clip=False)
# coaligned_maps10.peek();plt.colorbar();plt.show()

In [ ]:
coaligned_maps11 = align(smaps11)

# b_left = SkyCoord(*u.arcsec,  *u.arcsec, frame=smaps11[0].coordinate_frame)
# t_right = SkyCoord(*u.arcsec, *u.arcsec, frame=smaps11[0].coordinate_frame)
# template = smaps11[0].submap(bottom_left=b_left, top_right=t_right)
# coaligned_maps11 = co_align(smaps11,layer_index=0,template=template,clip=False)
# coaligned_maps11.peek();plt.colorbar();plt.show()

In [ ]:
# Now only after all the coalignment is done, we exposure normalise the data.
coaligned_maps1 = exp_norm(coaligned_maps1)
coaligned_maps2 = exp_norm(coaligned_maps2)
coaligned_maps3 = exp_norm(coaligned_maps3)
coaligned_maps4 = exp_norm(coaligned_maps4)
coaligned_maps5 = exp_norm(coaligned_maps5)
coaligned_maps6 = exp_norm(coaligned_maps6)
coaligned_maps7 = exp_norm(coaligned_maps7)
coaligned_maps8 = exp_norm(coaligned_maps8)
coaligned_maps9 = exp_norm(coaligned_maps9)
coaligned_maps10 = exp_norm(coaligned_maps10)
coaligned_maps11 = exp_norm(coaligned_maps11)


In [ ]:
# How to find the flare peak frame 

temp_lc = []

for smap in coaligned_maps3:
    data = smap.data
    temp_lc.append(np.nansum(data))
     
plt.figure()
plt.plot(np.arange(len(temp_lc)),temp_lc)
Plt.title('Total counts vs Frame number in NB03')    
plt.show()

In [ ]:
# Here we are taking a 60% contour during the peak of the flare in NB03 filter. 

j =
plt.figure()
contour_level = np.max(coaligned_maps3[j].data)*.6
contours = plt.contour(coaligned_maps3[j].data,levels=[contour_level],colors='red')
plt.imshow(coaligned_maps3[j].data,origin='lower')
plt.show()

In [ ]:
# creating a mask for the contour
path = contours.get_paths()[0]
data = coaligned_maps3[j].data
mask = np.zeros(data.shape, dtype=bool)
x, y = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
points = np.vstack((x.flatten(), y.flatten())).T 
mask = path.contains_points(points).reshape(data.shape)
plt.figure()
plt.imshow(mask,origin='lower')
plt.show()

In [ ]:
# This is to make contours on your maps and then save them as png. Choose power norm paramters according to how your image looks.

j= #flare peak in NB03 for this flare

contour_level1 = np.max(coaligned_maps3[j].data)*.3 
contour_level2 = np.max(coaligned_maps3[j].data)*.6

for i,smap in enumerate(coaligned_maps3):
    fig = plt.figure()
    ax = fig.add_subplot(projection=coaligned_maps3[0])
    ax.imshow(smap.data,cmap='suit_nb03',norm=PowerNorm(0.4,vmin=0,vmax=42000), origin='lower') #plotting just the image data
    
    # smap.plot(axes=ax,norm=PowerNorm(0.4,vmin=0000,vmax=42000), origin='lower')  #this one plots the map with WCS not just data
    # plt.colorbar() 
    
    c1 = plt.contour(coaligned_maps3[j].data,levels=[contour_level1],colors='red',linewidth=0.5,label='30%')  #making 30 percent contour from flare peak
    c2 = plt.contour(coaligned_maps3[j].data,levels=[contour_level2],colors='cyan',linewidth=0.5,label='60%') #making 60 percent contour from flare peak
    legend_elements = [Line2D([0], [0], color='red',  lw=2, label = '30% Contour'),
                       Line2D([0], [0], color='cyan', lw=2, label = '60% Contour')
                      ]
    ax.legend(handles=legend_elements, loc='upper left',bbox_to_anchor=(.9, 1.16),frameon=False)
   
    ax.grid(False)
    # break
    plt.savefig(f'images/{i:03d}.png',dpi=300)
    plt.close()

In [ ]:
# make_lc function makes lightcurves for all the filters by calculating mean of all the 
# unsaturated pixels inside the mask for the entire mapsequence.
time1,lc1 = make_lc(mask,coaligned_maps1)
time2,lc2 = make_lc(mask,coaligned_maps2)
time3,lc3 = make_lc(mask,coaligned_maps3)
time4,lc4 = make_lc(mask,coaligned_maps4)
time5,lc5 = make_lc(mask,coaligned_maps5)
time6,lc6 = make_lc(mask,coaligned_maps6)
time7,lc7 = make_lc(mask,coaligned_maps7)
time8,lc8 = make_lc(mask,coaligned_maps8)
time9,lc9 = make_lc(mask,coaligned_maps9)
time10,lc10 = make_lc(mask,coaligned_maps10)
time11,lc11 = make_lc(mask,coaligned_maps11)

In [ ]:
# loading the GOES file provided along with the dataset. This file should be in the same directory as this notebook or you can change the path accordingly.
file_goes = glob('*.nc')
goes_18 = ts.TimeSeries(file_goes)
xrs_short = goes_18.data['xrsb']
goes_18.peek()

In [ ]:
cb_colors = sns.color_palette("colorblind")
fig, ax = plt.subplots(4, 1, figsize=(10, 10), sharex=True)
ax = ax.flatten()

twin_axes = [a.twinx() for a in ax]

# --- GOES on regular axes ---
for i in range(4):
    ax[i].plot(xrs_short, color='red', label='GOES')   # red
    ax[i].set_ylim(1e-6, 4e-5)
    ax[i].set_yscale('log')
    ax[i].legend(frameon=False, loc=3)
    ax[i].axvline(x=time3[np.argmax(lc3)], color='black', linestyle='dotted')
    
# --- SUIT + BB data on twin axes ---
twin_axes[3].plot(time1, lc1/max(lc1), '.-', color=cb_colors[0], label='NB01')   # blue
twin_axes[1].plot(time2, lc2/max(lc2), '.-', color=cb_colors[9], label='NB02')   # orange
twin_axes[0].plot(time3, lc3/max(lc3), '.-.', color=cb_colors[2], label='NB03')  # green
twin_axes[0].plot(time4, lc4/max(lc4), 'h-', color=cb_colors[4], label='NB04')   # purple
twin_axes[1].plot(time5, lc5/max(lc5), '^-', color=cb_colors[4], label='NB05')   # brown
twin_axes[2].plot(time6, lc6/max(lc6), 's-', color=cb_colors[6], label='NB06')   # pink

# NB07 (separate twinx)
axnb7 = ax[2].twinx()
nb7col = cb_colors[7]      # grey
axnb7.plot(time7, lc7/max(lc7), '.-', color=nb7col, label='NB07', markersize=10)
axnb7.legend(frameon=False, loc=6)
axnb7.spines['right'].set_position(('outward', 40))
axnb7.tick_params(axis='y', colors=nb7col)
axnb7.spines['right'].set_color(nb7col)

twin_axes[0].plot(time8, lc8/max(lc8), 'h--', color=cb_colors[8], label='NB08')   # yellow
twin_axes[2].plot(time9, lc9/max(lc9), '--', color=cb_colors[9], marker='^', label='BB01')  # turquoise
twin_axes[3].plot(time10, lc10/max(lc10),'--',color=cb_colors[2] , label='BB02')

# BB03 (separate axis)
bb3col = cb_colors[1]  # orange
axbb3 = ax[3].twinx()
axbb3.plot(time11, lc11/max(lc11), '^-', color=bb3col, label='BB03')
axbb3.legend(frameon=False, loc=6)
axbb3.spines['right'].set_position(('outward', 40))
axbb3.tick_params(axis='y', colors=bb3col)
axbb3.spines['right'].set_color(bb3col)

ax[0].set_xlim(time3[0], time3[-1]+timedelta(minutes=5))

# Style main twin axes
for i in range(4):
    twin_axes[i].tick_params(axis='y', colors='black')
    twin_axes[i].spines['right'].set_color('black')
    twin_axes[i].legend(frameon=False, loc=2)

# # HEL1OS on ax[1]
# hcolor = cb_colors[2]  # green
# ax2 = ax[1].twinx()
# ax2.plot(th1, flux_smooth, '-', color=hcolor, label='HEL1OS 20–80 keV')
# ax2.tick_params(axis='y', colors=hcolor, 
#                 # labelsize=8, length=3, width=0.8
# )
# ax2.set_ylabel('HEL1OS Counts',fontsize=14,color = hcolor ,rotation=270,labelpad=15)
# ax2.spines['right'].set_color(hcolor)
# ax2.spines['right'].set_position(('outward', 40))
# ax2.set_yscale('log')
# ax2.legend(frameon=False, loc='best')

# Layout
fig.subplots_adjust(hspace=0.0)
fig.subplots_adjust(left=0.1, right=0.85, top=0.9, bottom=0.1)
fig.supxlabel('TIME (UT)', fontsize=16, y=0.04)
fig.text(0.01, 0.5, 'Flare Class W/m$^{2}$', rotation=90, va='center', fontsize=16)
plt.suptitle('60% Contour LightCurves ', fontsize=18, y=0.95)

# Global legend
vline_black = Line2D([0], [0], color='black', linestyle='dotted', label='NB03 Peak')
fig.legend(handles=[vline_black],
           loc='center right', frameon=False, fontsize=12,
           bbox_to_anchor=(.22, .92))

plt.show()


In [ ]:
# saving the lightcurve you just produced. We are using zip_longest because sometime the number of frames in different filters might not be the same. 
from itertools import zip_longest
with open('nblc_60contour.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['time1','lc1','time2','lc2','time3','lc3','time4','lc4','time5','lc5','time6','lc6','time7','lc7','time8','lc8'])   # Header  
    writer.writerows(zip_longest(time1, lc1, time2, lc2, time3, lc3, time4, lc4,time5, lc5, time6, lc6, time7, lc7, time8, lc8, fillvalue='')) # data